In [1]:
import torch
from torch import optim
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
import imp
import time
import os

import utils.movie_readin as mru
import utils.plotutils as plu
import utils.model as mod
#import utils.model_analysis as man
from torch.autograd import Variable

In [5]:
# movie parameters
frame_rate = 120
patch_size = 32
patch_seconds = 5
patch_frames = patch_seconds * frame_rate
movies_folder = '/data/stationary_motion/pixel2xlmomentlens/pngs'
patches_folder = f'/data/stationary_motion/pixel2xlmomentlens/patches/patches_{patch_size}px_{patch_seconds}s_{frame_rate}fps'

# model hyperparameter
lambda_activation = 0.1
lambda_biophysical = 0
conv_width = 8
hidden_nodes = 100
compression = patch_size**2 / hidden_nodes
print(f'Model:{patch_size}^2  = {patch_size**2} pixels by {patch_seconds} frames, to {hidden_nodes} hidden nodes for {compression}x compression')

# training parameters
num_epochs = 100
batch_sizes = [50]
learning_rates = [1e-3]
learning_momentum = 0.99
savefolder = './output/actv_'

Model:32^2  = 1024 pixels by 5 frames, to 100 hidden nodes for 10.24x compression


In [3]:
imp.reload(mru)
# create movie patches if we haven't yet
try:
    os.stat(patches_folder)
    print('Found Pre-computed natural movie patches. Loading them in...')
except:
    print('Couldn\'t find natual movie patches. Making them...')
    mru.createNatMoviePatches(framerate=frame_rate, patchsize=patch_size, seconds=patch_seconds,
                          read_folder=movies_folder, write_folder=patches_folder, patches_per_file=100000)
    print('Done Making Patches. Loading them in....')
#movie_dataset = mru.NaturalMovieDataset(patches_folder)
movie_dataset = mru.get_pkl_patch_movie(patches_folder)
print("Done!")

Found Pre-computed natural movie patches. Loading them in...
Done!


In [ ]:
imp.reload(mod)
imp.reload(mru)

for batch_size in batch_sizes:
    
    #train_loader = mru.DataLoader(movie_dataset, batch_size=batch_size,
    #                    shuffle=True, num_workers=8)
    # for now: read all movies in:
    #movies_as_array = movie_dataset.getallitems()
    
    for learning_rate in learning_rates:
        
        print(f'Learning Rate:{learning_rate}; batch size:{batch_size}')
        # our model
        model = mod.AEC(hidden_nodes, conv_width, patch_size, lambda_activation).double()
        #model = nn.DataParallel(model).cuda()
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        if torch.cuda.is_available():
            print('Found GPU - Running Model on it.')
            model.cuda()
        else:
            print('No GPUs found. Training on CPUs.')
        optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay = 0.5)#, momentum=learning_momentum)
        #optimizer = optim.SGD(model.parameters(), lr=learning_rate)#, momentum=learning_momentum)

        print_epocs = 20
        printing_modulo = num_epochs/print_epocs
        loss_history = []
        snr_history = []
        print(f'Training {num_epochs} Epochs. ',end="")
        # Because dataset is small and all in one .pkl file, it's faster to read all into memory 
        for i in range(num_epochs):
            start = time.time()
            times = []
            batches_start = np.arange(0,np.shape(movie_dataset)[0], batch_size)
            for bs in batches_start:
                movie_batch = torch.unsqueeze(torch.tensor(movie_dataset[bs:bs+batch_size]),1).cuda()          
                optimizer.zero_grad() #zero out our gradients
                acts = model.encode(movie_batch)
                recon_batch = model.decode(acts)
                loss = model.loss_func(movie_batch, recon_batch, acts)
                loss.backward()
                optimizer.step()
                end = time.time()
                times.append(end-start)
            # record loss and snr after each epoch
            loss_history.append(loss.item())
            snr_history.append(model.calc_snr(movie_batch, recon_batch).detach())
            # if it's our first iteration, 
            if(True):
                params = f'lr{learning_rate}_bs{batch_size}'
                if(i==0):
                    print(f'Estimated run time: {round(times[-1]*num_epochs/60,1)}mins.')
                elif((i+1)%printing_modulo==0):
                    print(f'Epoch {i+1}/{num_epochs} (mean time per epoch: {round(np.mean(times),1)}s)')
                    for name, parameter in model.named_parameters():
                        print(name)
                        if(name in ['tconv.module.weight_v', 'tconv.weight_v']):
                            inw = np.array(parameter.cpu().squeeze().detach())
                            break
                    p = plu.plot_temporal_weights(inw)
                    plt.savefig(f'{savefolder}inw_{params}_{round(i/num_epochs,2)}.png')
                    plt.show()
                else:
                    print('*',end='')

        loss_evolution = [np.float(loss.detach()) for loss in loss_history]
        snr_evolution =  [np.float(snr.detach()) for snr in snr_history]
        # plot evolution of loss and snr
        p = plt.plot(loss_evolution)
        plt.title('Loss Evolution')
        plt.savefig(f'{savefolder}loss_{params}.png')
        plt.clf()
        p = plt.plot(np.log(loss_evolution))
        plt.title('LogLoss Evolution')
        plt.savefig(f'{savefolder}logloss_{params}.png')
        plt.clf()
        p = plt.plot(snr_evolution)
        plt.title('SNR Evolution')
        plt.savefig(f'{savefolder}snr_{params}.png')
        plt.clf()

        for name, parameter in model.named_parameters():
            if(name in ['tconv.module.weight_v', 'tconv.weight_v']):
                inw = np.array(parameter.cpu().squeeze().detach())
            elif(name in ['tdeconv.module.weight', 'tdeconv.weight']):
                outw = np.array(parameter.cpu().squeeze().detach())
        
        #mp = model.named_parameters()
        #bias = np.array(next(mp).detach())
        #wnorm = next(mp)
        #inw = np.array(next(mp).squeeze().detach())
        #outw = np.array(next(mp).squeeze().detach())

        p = plu.plot_temporal_weights(inw)
        plt.savefig(f'{savefolder}inw_{params}.png')
        plt.clf()
        p = plu.plot_temporal_weights(outw)
        plt.savefig(f'{savefolder}outw_{params}.png')
        plt.clf()


        print('Done!')


Learning Rate:0.001; batch size:50
Found GPU - Running Model on it.
Training 100 Epochs. Estimated run time: 273.8mins.
*

In [ ]:
memReport()

In [ ]:
movie_dataset.shape

In [ ]:
torch.cuda.empty_cache()

In [ ]:
def debug_memory():
    import collections, gc, torch
    tensors = collections.Counter((str(o.device), o.dtype, tuple(o.shape))
                                  for o in gc.get_objects()
                                  if torch.is_tensor(o))
    for line in sorted(tensors.items()):
        print('{}\t{}'.format(*line))
debug_memory()



In [ ]:
loss_evolution = [np.float(loss.detach()) for loss in loss_history]
plt.plot(loss_evolution)
plt.show()
plt.plot(np.log(loss_evolution))
plt.show()
plt.plot(np.log(snr_history))
plt.show()

In [ ]:
for name, parameter in model.named_parameters():
    print(name)
    if(name=='tconv.module.weight_v'):
        inw = np.array(parameter.squeeze().detach().cpu())
    elif(name=='tdeconv.module.weight'):
        outw = np.array(parameter.squeeze().detach().cpu())
    #elif(name=='tconv.weight_g'):
    #    print(np.shape(np.array(parameter.squeeze().detach())))

#print(inw.shape)
#print(bias.shape)
#print(wnorm.shape)
#print(outw.shape)
p = plu.plot_temporal_weights(inw)
p = plu.plot_temporal_weights(outw)

In [ ]:
plt.hist(inw.flatten())
plt.hist(outw.flatten())

In [ ]:
imp.reload(plu)
#movies = movie_batch[0]
#recons = recon_batch[0]
print('Movies:')
for i in range(10):
    plu.plot_movies_recons(np.squeeze(movie_batch), np.squeeze(recon_batch), i)
    #plt.colorbar()
plt.show()


In [ ]:
outw

In [ ]:
print(inw.shape)
print(np.max(inw))
print(np.min(inw))
for i in range(inw.shape[1]):
    plt.imshow(inw[9,i,:,:],cmap='Greys_r')
    plt.show()

In [ ]:
print(movie.shape)
m = movie[7,0,1,:,:]
plt.imshow(m)
print(m)

In [ ]:
movie.shape

In [ ]:
if(False):
    moreepochs = 2000
    print(f'Training for {moreepochs} more Epochs:')
    for i in range(moreepochs):
        start = time.time()
        times = []
        for movie in train_loader:
            movie = torch.unsqueeze(movie,1)
            #print(movie.size())
            movie = movie.float().cuda()
            optimizer.zero_grad()
            acts = model.encode(movie)
            recon = model.decode(acts)
            loss = loss_func(movie, recon, acts)
            loss_history.append(loss.detach())
            loss.backward()
            optimizer.step()
            end=time.time()
            times.append(end-start)
            
        if((i+1)%printing_modulo==0):
            print(f'{i+1}th Epoch (mean time per epoch: {round(np.mean(times))}s)')
        else:
            print('*',end='')

    print('Done!')

In [ ]:
def visualize_aec():
    with torch.no_grad():
        # Get a batch of training data
        data = next(iter(train_loader))[0].to(device)

        input_tensor = data.cpu()
        transformed_input_tensor = model.encode(data).cpu()

        in_grid = convert_image_np(
            torchvision.utils.make_grid(input_tensor))

        out_grid = convert_image_np(
            torchvision.utils.make_grid(transformed_input_tensor))

        # Plot the results side-by-side
        f, axarr = plt.subplots(1, 2)
        axarr[0].imshow(in_grid)
        axarr[0].set_title('Dataset Images')

        axarr[1].imshow(out_grid)
        axarr[1].set_title('Recon Images')


visualize_aec()
plt.ioff()
plt.show()

In [ ]:
torchvision.utils.make_grid(input_tensor)

In [ ]:
# initialize figure
f, a = plt.subplots(2, N_TEST_IMG, figsize=(5, 2))
plt.ion()   # continuously plot

# original data (first row) for viewing
view_data = train_data.train_data[:N_TEST_IMG].view(-1, 28*28).type(torch.FloatTensor)/255.
for i in range(N_TEST_IMG):
    a[0][i].imshow(np.reshape(view_data.numpy()[i], (28, 28)), cmap='gray'); a[0][i].set_xticks(()); a[0][i].set_yticks(())

    
    
for epoch in range(EPOCH):
    for step, (x, b_label) in enumerate(train_loader):
        b_x = x.view(-1, 28*28)   # batch x, shape (batch, 28*28)
        b_y = x.view(-1, 28*28)   # batch y, shape (batch, 28*28)

        encoded, decoded = autoencoder(b_x)

        loss = loss_func(decoded, b_y)      # mean square error
        optimizer.zero_grad()               # clear gradients for this training step
        loss.backward()                     # backpropagation, compute gradients
        optimizer.step()                    # apply gradients

        if step % 100 == 0:
            print('Epoch: ', epoch, '| train loss: %.4f' % loss.numpy())

            # plotting decoded image (second row)
            _, decoded_data = autoencoder(view_data)
            for i in range(N_TEST_IMG):
                a[1][i].clear()
                a[1][i].imshow(np.reshape(decoded_data.numpy()[i], (28, 28)), cmap='gray')
                a[1][i].set_xticks(()); a[1][i].set_yticks(())
            plt.draw(); plt.pause(0.05)

plt.ioff()
plt.show()

In [ ]:
torch.cuda.empty_cache()
model = 'a'

In [ ]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 500 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
#
# A simple test procedure to measure STN the performances on MNIST.
#

def test():
    with torch.no_grad():
        model.eval()
        test_loss = 0
        correct = 0
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)

            # sum up batch loss
            test_loss += F.nll_loss(output, target, size_average=False).item()
            # get the index of the max log-probability
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()

        test_loss /= len(test_loader.dataset)
        print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'
              .format(test_loss, correct, len(test_loader.dataset),
                      100. * correct / len(test_loader.dataset)))
    

In [ ]:
model = AEC()
if torch.cuda.is_available():
    model.cuda()